In [ ]:
%%sql
-- HLL XM
CREATE DATABASE hll;

In [ ]:
%%sql
-- Deploy Extension
CREATE EXTENSION `hll-sketch` FROM HTTP 'https://github.com/singlestore-extensions/s2-hyperloglog-sketch-extension-cpp/raw/refs/heads/main/build/hll-sketch.tar';

In [ ]:
%%sql
DROP EXTENSION `hll-sketch`;

In [ ]:
%%sql
-- Verify Extension
SHOW EXTENSIONS;

In [ ]:
%%sql
-- Verify UDAF
SHOW AGGREGATES;

In [ ]:
%%sql
-- Verify UDF
SHOW FUNCTIONS;

In [ ]:
%%sql
-- Create a sample table
CREATE TABLE users (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    country VARCHAR(50)
);

In [ ]:
%%sql
-- Create a sample dataset
INSERT INTO users (id, name, country)
SELECT 
    10000 + (ROW_NUMBER() OVER ()),
    CONCAT('User', CAST(ROW_NUMBER() OVER () AS CHAR)),
    ELT(1 + FLOOR(RAND() * 3), 'USA', 'Canada', 'UK')
FROM 
    information_schema.columns
LIMIT 10000;

In [ ]:
%%sql
SELECT * FROM users LIMIT 5;

In [ ]:
%%sql
-- Test 1: hll_add_agg and hll_cardinality
SELECT 
    country,
    COUNT(DISTINCT id) AS actual_count,
    CAST(hll_cardinality(hll_add_agg(id)) AS UNSIGNED) AS estimated_count,
    CAST(hll_cardinality(hll_add_agg_compact(id)) AS UNSIGNED) AS estimated_count_compact,
    ABS(COUNT(DISTINCT id) - hll_cardinality(hll_add_agg(CAST(id AS BINARY)))) / COUNT(DISTINCT id) * 100 AS error_percentage
FROM 
    users
GROUP BY 
    country;

In [ ]:
%%sql
-- Test 2: hll_union and hll_cardinality
WITH country_sketches AS (
    SELECT 
        country,
        hll_add_agg(CAST(id AS BINARY)) AS sketch
    FROM 
        users
    GROUP BY 
        country
)
SELECT 
    'All Countries' AS country,
    (SELECT COUNT(DISTINCT id) FROM users) AS actual_total_count,
    CAST(hll_cardinality(hll_union(
        (SELECT sketch FROM country_sketches WHERE country = 'USA'),
        hll_union(
            (SELECT sketch FROM country_sketches WHERE country = 'Canada'),
            (SELECT sketch FROM country_sketches WHERE country = 'UK')
        )
    )) AS UNSIGNED) AS estimated_total_count,
    ABS((SELECT COUNT(DISTINCT id) FROM users) - hll_cardinality(hll_union(
        (SELECT sketch FROM country_sketches WHERE country = 'USA'),
        hll_union(
            (SELECT sketch FROM country_sketches WHERE country = 'Canada'),
            (SELECT sketch FROM country_sketches WHERE country = 'UK')
        )
    ))) / (SELECT COUNT(DISTINCT id) FROM users) * 100 AS error_percentage;

In [ ]:
%%sql
-- Test 3: Test serialization methods
SELECT 
    LENGTH(hll_add_agg(id)) AS serialized_length
FROM users;

In [ ]:
%%sql
-- Test 3: Test serialization methods
SELECT 
    LENGTH(hll_add_agg_compact(id)) AS compact_length
FROM users;

In [ ]:
%%sql
-- Test 4: hll_union_agg
WITH 
    hll_sketches AS (
        SELECT hll_add_agg(CAST(id AS BINARY)) AS sketch
        FROM users
    ),
    actual_count AS (
        SELECT COUNT(DISTINCT id) AS count 
        FROM users
    )
SELECT 
    'All Countries' AS country,
    actual_count.count AS actual_total_count,
    CAST(hll_cardinality(hll_union_agg(sketch)) AS UNSIGNED) AS estimated_total_count,
    ABS(actual_count.count - hll_cardinality(hll_union_agg(sketch))) / actual_count.count * 100 AS error_percentage
FROM 
    hll_sketches, actual_count;

In [ ]:
%%sql
-- Test 6: hll_print (for visual inspection)
SELECT 
    country,
    COUNT(DISTINCT id) AS actual_count,
    hll_print(hll_add_agg_compact(CAST(id AS BINARY))) AS hll_sketch_info
FROM 
    users
GROUP BY 
    country;

# Sparse Testing Approach

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS fruits (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50) NOT NULL
);

In [ ]:
%%sql
INSERT INTO fruits (name) VALUES
('apple'), ('banana'), ('cherry'), ('date'), ('elderberry'),
('fig'), ('grape'), ('honeydew'), ('imbe'), ('jackfruit'),
('kiwi'), ('lemon'), ('mango'), ('nectarine'), ('orange'),
('papaya'), ('quince'), ('raspberry'), ('strawberry'), ('tangerine');

In [ ]:
%%sql
SELECT * FROM fruits ORDER BY id;

In [ ]:
%%sql
-- Test 1: Create an HLL sketch from the fruits table
SELECT hll_add_agg(CAST(name AS BINARY)) AS hll_sketch FROM fruits;

In [ ]:
%%sql
-- Test 2: Get the cardinality estimate
SELECT 
    hll_cardinality(hll_add_agg(name)) AS estimated_count,
    COUNT(DISTINCT name) AS actual_count
    FROM fruits;

In [ ]:
%%sql
-- Test 3: Print the HLL sketch details
SELECT hll_print(hll_add_agg(CAST(name AS BINARY))) AS hll_details FROM fruits;

In [ ]:
%%sql
-- Test 4: Serialization methods
SELECT 
    LENGTH(hll_add_agg(CAST(name AS BINARY))) AS serialized_length
FROM fruits;

In [ ]:
%%sql
-- Test 4: Serialization methods
SELECT 
    LENGTH(hll_add_agg_compact(CAST(name AS BINARY))) AS compact_serialized_length
FROM fruits;

In [ ]:
%%sql
-- Test 5: Union operation
WITH 
    hll1 AS (SELECT hll_add_agg(CAST(name AS BINARY)) AS sketch FROM fruits WHERE id <= 2251799813685258),
    hll2 AS (SELECT hll_add_agg(CAST(name AS BINARY)) AS sketch FROM fruits WHERE id > 2251799813685258)
SELECT 
    hll_cardinality(hll1.sketch) AS hll1_count,
    hll_cardinality(hll2.sketch) AS hll2_count,
    hll_cardinality(hll_union(hll1.sketch, hll2.sketch)) AS union_count
FROM hll1, hll2;